#**Analisi dei flussi degli utenti Vodafone**

Lo studio dei flussi di persone all'interno delle aree urbane è di fondamentale importanza per comprendere al meglio le criticità emergenti nella mobilità locale e per esplorare le aree di potenziale miglioramento delle infrastrutture e dei trasporti locali.<br>
La mobilità degli utenti all'interno e verso Padova è stata monitorata utilizzando i dati forniti dal gestore di telefonia mobile Vodafone, che fornisce informazioni basate sulle connessioni degli utenti alle celle della rete.<br>
I dati forniti dal carrier comprendono il monitoraggio degli utenti connessi alla rete Vodafone di Padova in un periodo di quattro mesi, da febbraio a maggio 2018.<br>
Per fornire approfondimenti statistici sul numero e sul flusso di utenti, i dati sono stati aggregati in base all'origine e ai movimenti degli utenti, calcolando la media del numero di connessioni durante il periodo di monitoraggio.<br>
Per evitare ulteriori problemi di violazione della privacy, tutte le osservazioni con meno di 30 unità (ad esempio le aree giornaliere per le quali <30 utenti hanno contribuito) sono state scartate e/o accorpate in categorie dedicate (indicate con "altro").<br>

## **Set di dati** 

I dataset si trovano al seguente link: https://www.dropbox.com/scl/fo/jra3xy77imdqc6hjc8luu/h?dl=0&rlkey=3ui6k9bhie88zxsk70zs317ow . La cartella comprende parecchi dati, si invita ad anallizzarli adeguatamente alle esigenze del problema ed eventualmente scartare dati inutili ai fini del progetto. I dati sono in diversi formati ed alcuni di questi sono descritti di seguito.

* __day_od.csv__: tabella delle origini e delle destinazioni degli utenti mediate per giorno della settimana. I dati sono forniti con il dettaglio del mese, del tipo di utente (residente a Padova/visitatore italiano/visitatore straniero), del Paese di provenienza, insieme alla provincia e al comune dell'utente (se disponibili).
* __utenti_distinti_giorno.csv__: tabella del numero di utenti distinti per provenienza. I dati sono forniti con il dettaglio del mese, del tipo di utente (residente a Padova/visitatore italiano/visitatore straniero), del Paese di provenienza, insieme alla provincia e al comune dell'utente (se disponibili).

Le informazioni sono memorizzate nei campi secondo il seguente schema: 

- MONTH: mese analizzato
- DOW: giorno analizzato
- ORIGIN: area di provenienza dell'utente
- DESTINATION: area di destinazione dell'utente
- CUST_CLASS: tipo di utente (residente / visitatore italiano / visitatore straniero)
- COD_COUNTRY: codice paese dell'utente (es. 222=Italia)
- COD_PRO: codice della provincia dell'utente (ad es. 12=Varese) 
- PRO_COM: codice del comune dell'utente (es. 12026=Busto Arsizio)
- FLOW: numero di movimenti per una determinata data-ora (con un minimo di 30 utenti)
- VISITORS: numero complessivo di utenti 

Insieme ai file di dati, vengono fornite tre tabelle di ricerca che permettono di abbinare i codici ISTAT di paese, provincia e comune ai nomi reali.

* __codici_istat_comune.csv__: file di lookup contenente la mappatura dei codici ISTAT dei comuni
* __codici_istat_provincia.csv__: file di ricerca contenente la mappatura dei codici ISTAT della provincia
* __codici_nazioni.csv__: file di lookup contenente la mappatura del codice _paese_ con il suo nome

Ulteriori informazioni, utili per lo studio dei flussi di utenti, come il numero di abitanti di ogni provincia e la distanza tra Padova e tutte le altre province italiane, possono essere estratte sulla base dei dati raccolti dall'ISTAT (es. http://dati.istat.it/Index.aspx?lang=en&SubSessionId=bcd553a8-280a-4b08-afee-cf7a72fd2ad9, http://www.istat.it/storage/cartografia/matrici_distanze/Veneto.zip).


## Assegnazione

1. Preparazione dei dati: i file csv provengono da fonti diverse e quindi presentano differenze di codifica e di fine riga che devono essere prese in considerazione nella fase di preparazione dei dati. Assicurarsi che ogni file .csv sia interpretato correttamente.
* Classifica dei visitatori provenienti da paesi stranieri: in base al numero di visitatori totali per ogni paese, creare una classifica dei primi 20 paesi con il maggior numero di visitatori.
* Classifica dei visitatori italiani per provincia, ponderata per il numero di abitanti: in base al numero di visitatori totali per provincia italiana, creare una classifica delle prime 20 province con più visitatori tenendo conto del numero di abitanti.
* Studio dei flussi di visitatori: ipotizziamo di essere chiamati a fornire indicazioni su come investire risorse per migliorare la mobilità verso Padova. Definite le tre principali direzioni dei visitatori e dei pendolari che raggiungono Padova attraverso le autostrade italiane (sud: A13 verso Bologna-Roma / ovest: A4 verso Milano-Torino / nord-est: A4 verso Venezia-Trieste), valutare quale delle tre direzioni deve essere privilegiata:
 1. Considerare un caso semplificato che coinvolge solo la mobilità di fascia media, in base al numero di visitatori/pendolari provenienti solo dalle regioni vicine
 2. Considerare le province situate sulle tre direzioni che contribuiscono maggiormente al flusso di visitatori del fine settimana e di pendolari giornalieri, effettuando uno studio più dettagliato dei flussi in base al giorno della settimana.
 3. Utilizzate i dati disponibili per fornire quella che ritenete la migliore risposta possibile. 
* Tracciare la distribuzione del numero di visitatori in base alla distanza della provincia di origine. Determinare il tipo di legge da utilizzare per descrivere la distribuzione.
 * Supponendo che si possa usare una forma analitica per descrivere l'andamento, create una regressione per stimare il numero atteso di visitatori in base alla distanza della provincia di origine. Illustrare la differenza tra la regressione ottenuta e i numeri forniti dal monitoraggio Vodafone, evidenziando le cinque discrepanze più evidenti rispetto alle aspettative.
* Utilizzate tutti i dati disponibili (ed eventualmente quelli aggiuntivi di fonte ISTAT) per estrarre un'altra informazione interessante sulla mobilità e sul flusso di visitatori e pendolari.

In [54]:
import pandas as pd
import numpy as np

In [55]:
df_day = pd.read_csv("DATA/day_od.csv")
df_distinct_users = pd.read_csv("DATA/distinct_users_day.csv", encoding="ANSI")
df_provincie = pd.read_csv("DATA/codici_istat_provincia.csv", encoding="ANSI")
df_comuni = pd.read_csv("DATA/codici_istat_comune.csv", encoding="ANSI")
df_nazioni = pd.read_csv("DATA/codici_nazioni.csv", encoding="ANSI")
df_population_data = pd.read_csv("DATA/DCIS_POPRES1_28032023144458463.csv")
df_population_data = df_population_data.loc[:, ["Territorio", "Value"]]
df_population_data.drop(11, inplace=True) # rimuovo il doppione di valle d'aosta

#df_day[(df_day.MONTH == "Marzo") & (df_day.DOW == "Martedì")]


In [56]:
df_population_data

,Territorio,Value
0,Italia,59030133
1,Nord-ovest,15831941
2,Piemonte,4256350
3,Torino,2208370
4,Vercelli,166083
...,...,...
130,Agrigento,415887
131,Siracusa,385900
132,Oristano,151655
133,Monza e della Brianza,870407


In [57]:
df_provincie

,COD_REG,COD_PRO,PROVINCIA,PROV_SIGLA
0,1,1,Torino,TO
1,1,2,Vercelli,VC
2,1,3,Novara,NO
3,1,4,Cuneo,CN
4,1,5,Asti,AT
...,...,...,...,...
103,20,91,Nuoro,NU
104,20,92,Cagliari,CA
105,20,95,Oristano,OR
106,20,111,Sud Sardegna,SU


In [58]:
df_distinct_users["PROVINCIA"] = df_distinct_users["COD_PRO"].map(df_provincie.set_index("COD_PRO")["PROVINCIA"])
df_distinct_users["COD_REG"] = df_distinct_users["COD_PRO"].map(df_provincie.set_index("COD_PRO")["COD_REG"])
df_distinct_users["COMUNE"] = df_distinct_users["PRO_COM"].map(df_comuni.set_index("PRO_COM")["COMUNE"])
df_distinct_users["COUNTRY"] = df_distinct_users["COD_COUNTRY"].map(df_nazioni.set_index("COD_COUNTRY")["COUNTRY_NAME_IT"])
df_distinct_users["VALUE"] = df_distinct_users["PROVINCIA"].map(df_population_data.set_index("Territorio")["Value"])
df_distinct_users

,DOW,CUST_CLASS,COD_COUNTRY,COD_PRO,PRO_COM,VISITORS,PROVINCIA,COD_REG,COMUNE,COUNTRY,VALUE
0,Mercoledì,visitor,222.0,35.0,35033.0,968,Reggio nell'Emilia,8.0,Reggio nell'Emilia,Italia,525586.0
1,Lunedì,visitor,222.0,22.0,22098.0,64,Trento,4.0,Isera,Italia,540958.0
2,Domenica,visitor,222.0,52.0,52032.0,516,Siena,9.0,Siena,Italia,261209.0
3,Giovedì,visitor,222.0,108.0,108009.0,128,Monza e della Brianza,3.0,Biassono,Italia,870407.0
4,Giovedì,visitor,222.0,29.0,29048.0,512,Rovigo,5.0,Villadose,Italia,228902.0
...,...,...,...,...,...,...,...,...,...,...,...
12840,Mercoledì,foreigner,259.0,NaN,NaN,176,NaN,NaN,NaN,Moldova,NaN
12841,Sabato,foreigner,602.0,NaN,NaN,164,NaN,NaN,NaN,Egitto,NaN
12842,Giovedì,foreigner,732.0,NaN,NaN,80,NaN,NaN,NaN,Colombia,NaN
12843,Giovedì,foreigner,297.0,NaN,NaN,80,NaN,NaN,NaN,Montenegro,NaN


In [59]:
venezia_trieste_A4 = [5, 6]
torino_milano_A4 = [1, 2, 3, 4, 7]
bologna_roma_A4 = [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
#df_venezia_trieste_A4_provincie = [df_provincie[df_provincie.COD_REG == x].PROVINCIA for x in venezia_trieste_A4]
#df_torino_milano_A4_provincie = [df_provincie[df_provincie.COD_REG == x].PROVINCIA for x in torino_milano_A4]
#df_bologna_roma_A4_provincie = [df_provincie[df_provincie.COD_REG == x].PROVINCIA for x in bologna_roma_A4]

In [60]:
#df_distinct_users["AUTOSTRADA"] = df_distinct_users["COD_REG"](venezia_trieste_A4)

In [61]:
#df_distinct_users["AUTOSTRADA"] = df_distinct_users["COD_REG"]
df_distinct_users["AUTOSTRADA"] = df_distinct_users.loc[:, "COD_REG"]

In [62]:
df_distinct_users["AUTOSTRADA"] = df_distinct_users["AUTOSTRADA"].map(lambda x : 0 if x in venezia_trieste_A4 else x)
df_distinct_users["AUTOSTRADA"] = df_distinct_users["AUTOSTRADA"].map(lambda x : 1 if x in torino_milano_A4 else x)
df_distinct_users["AUTOSTRADA"] = df_distinct_users["AUTOSTRADA"].map(lambda x : 2 if x in bologna_roma_A4 else x)

In [63]:
df_grouped_by_road = df_distinct_users.groupby(by=["AUTOSTRADA", "DOW"]).agg({"VISITORS": "sum", "COD_REG": "count"})
df_grouped_by_road

VISITORS  COD_REG
AUTOSTRADA DOW                         
0.0        Domenica     720796      578
           Giovedì      694828      580
           Lunedì       695136      572
           Martedì      702304      578
           Mercoledì    719856      583
           Sabato       740608      585
           Venerdì      725392      589
1.0        Domenica      84732      606
           Giovedì       73824      558
           Lunedì        71724      558
           Martedì       72608      566
           Mercoledì     76292      583
           Sabato        84904      615
           Venerdì       84820      609
2.0        Domenica      97900      609
           Giovedì       76848      548
           Lunedì        80312      580
           Martedì       80036      587
           Mercoledì     81688      604
           Sabato        98132      591
           Venerdì       89020      590

In [96]:
df_grouped_by_road.reset_index()

,AUTOSTRADA,DOW,VISITORS,COD_REG
0,0.0,Domenica,720796,578
1,0.0,Giovedì,694828,580
2,0.0,Lunedì,695136,572
3,0.0,Martedì,702304,578
4,0.0,Mercoledì,719856,583
5,0.0,Sabato,740608,585
6,0.0,Venerdì,725392,589
7,1.0,Domenica,84732,606
8,1.0,Giovedì,73824,558
9,1.0,Lunedì,71724,558


In [95]:
df_grouped_by_road.index[0][1]

'Domenica'

In [101]:
giorni_ordinati = ["Lunedì", "Martedì" , "Mercoledì" , "Giovedì" , "Venerdì" , "Sabato" , "Domenica"]
#df_grouped_by_road.reset_index().sort_values(by=["DOW"], key=lambda x: x.map(giorni_ordinati.index)).groupby(by=["AUTOSTRADA", "DOW"]).sum()

VISITORS  COD_REG
AUTOSTRADA DOW                         
0.0        Domenica     720796      578
           Giovedì      694828      580
           Lunedì       695136      572
           Martedì      702304      578
           Mercoledì    719856      583
           Sabato       740608      585
           Venerdì      725392      589
1.0        Domenica      84732      606
           Giovedì       73824      558
           Lunedì        71724      558
           Martedì       72608      566
           Mercoledì     76292      583
           Sabato        84904      615
           Venerdì       84820      609
2.0        Domenica      97900      609
           Giovedì       76848      548
           Lunedì        80312      580
           Martedì       80036      587
           Mercoledì     81688      604
           Sabato        98132      591
           Venerdì       89020      590

In [64]:
df_distinct_users[df_distinct_users["COD_PRO"] == 35.0].VISITORS.sum()

16320

In [65]:
df_distinct_users[df_distinct_users["COD_PRO"] == 35.0].VALUE[0]

525586.0

In [66]:
df_distinct_users[df_distinct_users["COD_PRO"] == 105.0].VISITORS.sum() / df_distinct_users[df_distinct_users["COD_PRO"] == 105.0].VALUE.iloc[0]

nan

In [67]:
dict(df_distinct_users.COD_PRO.value_counts())

{24.0: 671,
 28.0: 650,
 26.0: 610,
 15.0: 593,
 23.0: 566,
 22.0: 495,
 17.0: 421,
 30.0: 381,
 16.0: 363,
 27.0: 302,
 25.0: 298,
 29.0: 297,
 20.0: 285,
 12.0: 275,
 37.0: 253,
 108.0: 245,
 21.0: 239,
 1.0: 222,
 63.0: 219,
 36.0: 201,
 93.0: 198,
 58.0: 193,
 72.0: 180,
 38.0: 166,
 35.0: 149,
 48.0: 144,
 13.0: 126,
 87.0: 109,
 34.0: 105,
 42.0: 101,
 54.0: 98,
 65.0: 93,
 50.0: 93,
 3.0: 92,
 18.0: 91,
 39.0: 89,
 10.0: 83,
 75.0: 81,
 78.0: 76,
 97.0: 76,
 40.0: 72,
 71.0: 72,
 19.0: 70,
 52.0: 67,
 46.0: 66,
 47.0: 66,
 59.0: 66,
 31.0: 65,
 9.0: 63,
 67.0: 61,
 60.0: 61,
 73.0: 61,
 43.0: 61,
 41.0: 59,
 69.0: 58,
 61.0: 58,
 51.0: 57,
 99.0: 54,
 6.0: 53,
 49.0: 48,
 74.0: 47,
 92.0: 46,
 4.0: 44,
 110.0: 43,
 14.0: 42,
 98.0: 39,
 33.0: 38,
 68.0: 35,
 64.0: 35,
 81.0: 34,
 109.0: 33,
 11.0: 33,
 44.0: 33,
 103.0: 32,
 88.0: 32,
 104.0: 29,
 80.0: 29,
 76.0: 28,
 83.0: 28,
 2.0: 28,
 32.0: 27,
 8.0: 27,
 96.0: 27,
 77.0: 26,
 53.0: 24,
 100.0: 23,
 66.0: 23,
 70.0: 22,
 89

In [68]:
df_distinct_users = df_distinct_users.dropna(subset=["COD_PRO"])

In [69]:
lista = []
for code in df_distinct_users.COD_PRO.unique():
    lista.append((df_distinct_users[df_distinct_users["COD_PRO"] == code].VISITORS.sum() / df_distinct_users[df_distinct_users["COD_PRO"] == code].VALUE.iloc[0] , df_distinct_users[df_distinct_users["COD_PRO"] == code].PROVINCIA.iloc[0]))

In [90]:
df_rapporti = pd.DataFrame(lista , columns=["Rapporto", "PROVINCIA"])
# capire i nan
df_rapporti.isna().sum()

Rapporto     5
PROVINCIA    3
dtype: int64

In [71]:
# classifica delle prime 20 province per rapporto tra visitatori e popolazione
df_rapporti.sort_values(by="Rapporto", ascending=False).reset_index(drop=True).head(20)

,Rapporto,PROVINCIA
0,3.364067,Padova
1,0.741097,Venezia
2,0.714122,Rovigo
3,0.506903,Vicenza
4,0.361348,Treviso
5,0.236526,Belluno
6,0.196536,Verona
7,0.158529,Ferrara
8,0.123668,Pordenone
9,0.120431,Trento


In [72]:
df_distinct_users[df_distinct_users["COD_PRO"] == 26.0]

,DOW,CUST_CLASS,COD_COUNTRY,COD_PRO,PRO_COM,VISITORS,PROVINCIA,COD_REG,COMUNE,COUNTRY,VALUE,AUTOSTRADA
20,Sabato,visitor,222.0,26.0,26006.0,244,Treviso,5.0,Caerano di San Marco,Italia,877890.0,0.0
21,Lunedì,visitor,222.0,26.0,26006.0,304,Treviso,5.0,Caerano di San Marco,Italia,877890.0,0.0
29,Martedì,visitor,222.0,26.0,26083.0,268,Treviso,5.0,Susegana,Italia,877890.0,0.0
39,Domenica,visitor,222.0,26.0,26056.0,320,Treviso,5.0,Pederobba,Italia,877890.0,0.0
48,Venerdì,visitor,222.0,26.0,26088.0,208,Treviso,5.0,Vazzola,Italia,877890.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
12712,Martedì,visitor,222.0,26.0,26014.0,220,Treviso,5.0,Cavaso del Tomba,Italia,877890.0,0.0
12732,Lunedì,visitor,222.0,26.0,26009.0,588,Treviso,5.0,Casale sul Sile,Italia,877890.0,0.0
12746,Giovedì,visitor,222.0,26.0,26019.0,200,Treviso,5.0,Codognè,Italia,877890.0,0.0
12765,Venerdì,visitor,222.0,26.0,26039.0,136,Treviso,5.0,Maser,Italia,877890.0,0.0


In [73]:
"""i = 0
for code in df_distinct_users.dropna(subset=["COD_PRO"])["COD_PRO"].unique():
    print(df_distinct_users.dropna(subset=["COD_PRO"])[df_distinct_users.dropna(subset=["COD_PRO"])["COD_PRO"] == code].VISITORS.sum() / df_distinct_users.dropna(subset=["COD_PRO"])[df_distinct_users.dropna(subset=["COD_PRO"])["COD_PRO"] == code].VALUE.iloc[0])
    print(i)
    i += 1"""

'i = 0\nfor code in df_distinct_users.dropna(subset=["COD_PRO"])["COD_PRO"].unique():\n    print(df_distinct_users.dropna(subset=["COD_PRO"])[df_distinct_users.dropna(subset=["COD_PRO"])["COD_PRO"] == code].VISITORS.sum() / df_distinct_users.dropna(subset=["COD_PRO"])[df_distinct_users.dropna(subset=["COD_PRO"])["COD_PRO"] == code].VALUE.iloc[0])\n    print(i)\n    i += 1'

In [74]:
df_distinct_users.dropna(subset=["COD_PRO"])["COD_PRO"]

0         35.0
1         22.0
2         52.0
3        108.0
4         29.0
         ...  
12828     65.0
12829     22.0
12830     16.0
12831     78.0
12832     38.0
Name: COD_PRO, Length: 12312, dtype: float64

In [75]:
df_distinct_users

,DOW,CUST_CLASS,COD_COUNTRY,COD_PRO,PRO_COM,VISITORS,PROVINCIA,COD_REG,COMUNE,COUNTRY,VALUE,AUTOSTRADA
0,Mercoledì,visitor,222.0,35.0,35033.0,968,Reggio nell'Emilia,8.0,Reggio nell'Emilia,Italia,525586.0,2.0
1,Lunedì,visitor,222.0,22.0,22098.0,64,Trento,4.0,Isera,Italia,540958.0,1.0
2,Domenica,visitor,222.0,52.0,52032.0,516,Siena,9.0,Siena,Italia,261209.0,2.0
3,Giovedì,visitor,222.0,108.0,108009.0,128,Monza e della Brianza,3.0,Biassono,Italia,870407.0,1.0
4,Giovedì,visitor,222.0,29.0,29048.0,512,Rovigo,5.0,Villadose,Italia,228902.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
12828,Martedì,visitor,222.0,65.0,65052.0,36,Salerno,15.0,Fisciano,Italia,1064493.0,2.0
12829,Giovedì,visitor,222.0,22.0,22017.0,40,Trento,4.0,Bleggio Superiore,Italia,540958.0,1.0
12830,Martedì,visitor,222.0,16.0,16053.0,36,Bergamo,3.0,Caravaggio,Italia,1102997.0,1.0
12831,Venerdì,visitor,222.0,78.0,78031.0,36,Cosenza,18.0,Castrolibero,Italia,674543.0,2.0


In [76]:
df_distinct_users.COD_PRO.unique()

array([ 35.,  22.,  52., 108.,  29.,  27.,  89.,  20.,  28.,  98.,  24.,
        26.,  12.,  15.,  39.,  72.,  37.,  19.,  62.,  25.,  23.,   3.,
        14.,   1.,  81.,  92.,  31.,  78.,  17.,  93.,  58.,  16.,  21.,
        71.,  34.,  83.,   8., 104.,  42.,  73.,   9.,  36.,  97.,  30.,
        13.,  38.,  55.,  68.,   7., 103.,  18.,  10.,  41.,  67.,  87.,
       110., 109.,  50.,  32.,  63.,  60.,   6.,  77.,  82.,  65.,  46.,
        74.,  76.,  40.,  75.,   4.,  59.,  99.,  61.,  54.,  69.,  48.,
        90.,  45., 100.,  96.,  80.,  95.,  43.,  49.,  44.,  70.,  64.,
        88.,  47.,   2.,  53.,  66.,  79.,  56.,  11., 107.,  51.,  86.,
        33.,   5.,  85.,  84., 101.,  91., 102.,  57., 105.])

In [77]:
##replace nation number with string names
#nation_dictionary = {key: val for key, val in zip(df_nazioni.COD_COUNTRY, df_nazioni.COUNTRY_NAME_IT)}
#provincie_dictionary = {key: val for key, val in zip(df_provincie.COD_PRO, df_provincie.PROVINCIA)}
#comuni_dictionary = {key: val for key, val in zip(df_comuni.PRO_COM, df_comuni.COMUNE)}
#
#df_day.COUNTRY = df_day.COD_COUNTRY.replace(nation_dictionary)
#df_day.PRO = df_day.COD_PRO.replace(provincie_dictionary)
#df_day.COM = df_day.PRO_COM.replace(comuni_dictionary)
#
#df_distinct_users.COD_COUNTRY = df_distinct_users.COD_COUNTRY.replace(nation_dictionary)
#df_distinct_users.COD_PRO = df_distinct_users.COD_PRO.replace(provincie_dictionary)
#df_distinct_users.PRO_COM = df_distinct_users.PRO_COM.replace(comuni_dictionary)
#df_distinct_users

In [78]:
#primi 20 paesi stranieri con più visitatori
df_foreign_data = df_distinct_users[df_distinct_users.COUNTRY != "Italia"]
grouped_df_foreign_data = df_foreign_data.groupby(by="COUNTRY").sum().sort_values("VISITORS", ascending=False).iloc[:20, :]
grouped_df_foreign_data

,COD_COUNTRY,COD_PRO,PRO_COM,VISITORS,COD_REG,VALUE,AUTOSTRADA
COUNTRY,,,,,,,


In [79]:
"""grouped_df_foreign_data.reset_index(inplace=True)
grouped_df_foreign_data
grouped_df_foreign_data["COD_COUNTRY"] = grouped_df_foreign_data["COD_COUNTRY"].apply(lambda x: df_nazioni[df_nazioni.COD_COUNTRY == x].COUNTRY_NAME_IT.values[0])
grouped_df_foreign_data"""

'grouped_df_foreign_data.reset_index(inplace=True)\ngrouped_df_foreign_data\ngrouped_df_foreign_data["COD_COUNTRY"] = grouped_df_foreign_data["COD_COUNTRY"].apply(lambda x: df_nazioni[df_nazioni.COD_COUNTRY == x].COUNTRY_NAME_IT.values[0])\ngrouped_df_foreign_data'

In [80]:
df_distinct_users[df_distinct_users.PROVINCIA == "Torino"]

,DOW,CUST_CLASS,COD_COUNTRY,COD_PRO,PRO_COM,VISITORS,PROVINCIA,COD_REG,COMUNE,COUNTRY,VALUE,AUTOSTRADA
43,Mercoledì,visitor,222.0,1.0,1171.0,60,Torino,1.0,Orbassano,Italia,2208370.0,1.0
105,Martedì,visitor,222.0,1.0,1292.0,64,Torino,1.0,Venaria Reale,Italia,2208370.0,1.0
285,Martedì,visitor,222.0,1.0,1192.0,40,Torino,1.0,Pino Torinese,Italia,2208370.0,1.0
302,Mercoledì,visitor,222.0,1.0,1059.0,68,Torino,1.0,Carmagnola,Italia,2208370.0,1.0
333,Lunedì,visitor,222.0,1.0,1214.0,48,Torino,1.0,Rivalta di Torino,Italia,2208370.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
12657,Domenica,visitor,222.0,1.0,1082.0,108,Torino,1.0,Chivasso,Italia,2208370.0,1.0
12673,Lunedì,visitor,222.0,1.0,1272.0,3260,Torino,1.0,Torino,Italia,2208370.0,1.0
12757,Giovedì,visitor,222.0,1.0,1164.0,120,Torino,1.0,Nichelino,Italia,2208370.0,1.0
12807,Mercoledì,visitor,222.0,1.0,1082.0,52,Torino,1.0,Chivasso,Italia,2208370.0,1.0


In [81]:
"""df_population_data = pd.read_csv("DATA/DCIS_POPRES1_28032023144458463.csv")
df_population_data = df_population_data.loc[:, ["Territorio", "Value"]]
df_provincie"""

'df_population_data = pd.read_csv("DATA/DCIS_POPRES1_28032023144458463.csv")\ndf_population_data = df_population_data.loc[:, ["Territorio", "Value"]]\ndf_provincie'

In [82]:
"""df_distance_from_padova = df_distance_from_padova[df_distance_from_padova.Destinazione == "28060,00"]
df_distance_from_padova
df_comuni"""

'df_distance_from_padova = df_distance_from_padova[df_distance_from_padova.Destinazione == "28060,00"]\ndf_distance_from_padova\ndf_comuni'

In [83]:
venezia_trieste_A4 = [5, 6]
df = [df_provincie[df_provincie.COD_REG == x].PROVINCIA for x in range(5, 7)]
torino_milano_A4 = [1, 2, 3, 4, 7]
bologna_roma_A4 = [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [84]:
df_day

,MONTH,DOW,ORIGIN,DESTINATION,CUST_CLASS,COD_COUNTRY,COD_PRO,PRO_COM,FLOW
0,Marzo,Domenica,108,300,visitor,222,28.0,-999.0,493
1,Maggio,Lunedì,300,101,visitor,222,93.0,-999.0,58
2,Febbraio,Sabato,108,207,visitor,222,28.0,-999.0,39
3,Aprile,Venerdì,109,121,resident,222,28.0,28060.0,106
4,Maggio,Giovedì,300,120,visitor,222,28.0,28103.0,136
...,...,...,...,...,...,...,...,...,...
193496,Febbraio,Martedì,205,120,Altro,-999,-999.0,-999.0,32
193497,Marzo,Giovedì,214,115,Altro,-999,-999.0,-999.0,32
193498,Febbraio,Giovedì,124,121,Altro,-999,-999.0,-999.0,40
193499,Aprile,Domenica,110,300,visitor,222,28.0,28086.0,36


In [85]:
df_provincie[df_provincie.COD_REG == 1].PROVINCIA

0                  Torino
1                Vercelli
2                  Novara
3                   Cuneo
4                    Asti
5             Alessandria
6                  Biella
7    Verbano-Cusio-Ossola
Name: PROVINCIA, dtype: object